In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import yfinance as yf

In [29]:
# For fetching financial data  
class DataFetcher:
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
    def fetch_data(self):
            data = yf.download(self.ticker, start=self.start_date, end=self.end_date)
            return data

In [32]:
class FeatureEngineer:
    def __init__(self, data):
        self.data = data
    def create_features(self):
            # Example features 
            self.data['SMA_20'] = self.data['Close'].rolling(window=20).mean()
            self.data['RSI'] = self.calculate_rsi()
            return self.data.dropna()
    def calculate_rsi(self, period=14):
            delta = self.data['Close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            return 100 - (100 / (1 + rs))

In [36]:
class Model:
    def __init__(self):
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.scaler = StandardScaler()
    def train(self, X, y):
            X_scaled = self.scaler.fit_transform(X)
            X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
            self.model.fit(X_train, y_train)
            predictions = self.model.predict(X_test)
            mse = mean_squared_error(y_test, predictions)
            return mse
    def predict(self, X):
            X_scaled = self.scaler.transform(X)
            return self.model.predict(X_scaled)

In [38]:
class Strategy:
    def __init__(self, model, threshold=0.5):
        self.model = model
        self.threshold = threshold
    def make_decision(self, features):
        prediction = self.model.predict(features.reshape(1, -1))
        if prediction > self.threshold:
            return "Buy"
        elif prediction < -self.threshold:
             return "Sell"
        else:
             return prediction

In [27]:
class HedgeFundTeam:
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.data_fetcher = DataFetcher(ticker, start_date, end_date)
        self.feature_engineer = None
        self.model = Model()
        self.strategy = None
    def run(self):
            data = self.data_fetcher.fetch_data()
            self.feature_engineer = FeatureEngineer(data)
            features = self.feature_engineer.create_features()
            X = features[['SMA_20', 'RSI']].values
            y = features['Close'].pct_change().shift(-1).fillna(0).values
            mse = self.model.train(X, y)
            print(f"Model Mean Squared Error: {mse}")
            self.strategy = Strategy(self.model)
            # Simulate trading decisions for the last few days 
            for i in range(-10, 0):
                # Last 10 days
                decision = self.strategy.make_decision(X[i])
                print(f"Day {i}: Decision - {decision}")          

In [39]:
# Usage 
team = HedgeFundTeam('AAPL', '2021-01-01', '2025-01-24')
team.run()

[*********************100%***********************]  1 of 1 completed
C:\Users\Maansi\AppData\Local\Temp\ipykernel_12528\828837541.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model.fit(X_train, y_train)


Model Mean Squared Error: 0.00034994414310572387
Day -10: Decision - [-0.01760325]
Day -9: Decision - [-0.01077561]
Day -8: Decision - [-0.00844076]
Day -7: Decision - [0.00524762]
Day -6: Decision - [-0.02248565]
Day -5: Decision - [-0.017691]
Day -4: Decision - [-0.01946815]
Day -3: Decision - [2.33105525e-05]
Day -2: Decision - [0.00851117]
Day -1: Decision - [-0.00301645]
